In [1]:
import numpy as np
import pickle
import glob
import matplotlib.pyplot as plt
import pandas as pd
from scipy import *
import os
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn import *
from sklearn.metrics import *
sns.set()
import warnings
warnings.filterwarnings("ignore")
length = 277

2024-03-19 20:34:52.187753: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-19 20:34:52.209946: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-19 20:34:52.209962: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-19 20:34:52.210675: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-19 20:34:52.215115: I tensorflow/core/platform/cpu_feature_guar

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9572379679743641107
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5803868160
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1589391405304624784
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9"
xla_global_id: 416903419
]


2024-03-19 20:34:55.045851: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-19 20:34:55.174540: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-19 20:34:55.174581: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-19 20:34:55.461194: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-19 20:34:55.461289: I external/local_xla/xla/stream_executor

In [3]:
train_values = pd.read_csv('./data/train_test/train_patients.csv').values
test_values = pd.read_csv('./data/train_test/test_patients.csv').values

print(train_values.shape)
print(test_values.shape)


# Separate the training and testing data, and one-hot encode Y:
X_train = train_values[:,:-2]
X_test = test_values[:,:-2]
y_train = train_values[:,-2]
y_test = test_values[:,-2]
X_test1 = X_test.reshape(-1, X_train.shape[1], 1)
y_test1=to_categorical(y_test)


X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)
X_train1 = X_train.reshape(-1, X_train.shape[1], 1)
y_train1=to_categorical(y_train)

X_val1 = X_val.reshape(-1, X_train.shape[1], 1)
y_val1=to_categorical(y_val)


(38751, 277)
(72794, 277)
X_train shape: (31000, 275)
X_val shape: (7751, 275)
y_train shape: (31000,)
y_val shape: (7751,)


In [4]:
from tensorflow.keras.layers import Activation, BatchNormalization

def mlpmodel():
    mlpmodel = Sequential()
    mlpmodel.add(Dense(275, input_shape=(X_train.shape[1],)))
    mlpmodel.add(Dense(50))
    mlpmodel.add(BatchNormalization())
    mlpmodel.add(Activation('relu'))
    mlpmodel.add(Dense(150))
    mlpmodel.add(BatchNormalization())
    mlpmodel.add(Activation('relu'))
    mlpmodel.add(Dense(900))
    mlpmodel.add(BatchNormalization())
    mlpmodel.add(Activation('relu'))
    mlpmodel.add(Dense(400))
    mlpmodel.add(BatchNormalization())
    mlpmodel.add(Activation('relu'))
    mlpmodel.add(Dense(7, activation='softmax'))
    
    return mlpmodel

mlp = mlpmodel()
mlp.summary()

2024-03-19 20:34:57.167211: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-19 20:34:57.167281: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-19 20:34:57.167298: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-19 20:34:57.167512: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-19 20:34:57.167532: I external/local_xla/xla/stream_executor

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 275)               75900     
                                                                 
 dense_1 (Dense)             (None, 50)                13800     
                                                                 
 batch_normalization (Batch  (None, 50)                200       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 50)                0         
                                                                 
 dense_2 (Dense)             (None, 150)               7650      
                                                                 
 batch_normalization_1 (Bat  (None, 150)               600       
 chNormalization)                                       

In [5]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)


In [6]:
class_weights

array([0.25901974, 2.27306057, 2.33468896, 2.37438725, 2.32942591,
       2.37766529])

In [7]:
mlp.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, mode='min',min_delta=0.001, restore_best_weights=True)
history = mlp.fit(X_train, y_train1, validation_data=(X_val, y_val1), epochs=50,callbacks=[early_stopping], verbose=1)


Epoch 1/50


2024-03-19 20:34:58.478531: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-19 20:34:58.717055: I external/local_xla/xla/service/service.cc:168] XLA service 0x556549c51c80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-19 20:34:58.717084: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2024-03-19 20:34:58.730373: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-19 20:34:58.756092: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1710880498.809568  267198 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


969/969 [==============================] - 10s 8ms/step - loss: 0.2125 - accuracy: 0.9333 - val_loss: 0.1430 - val_accuracy: 0.9587
Epoch 2/50
969/969 [==============================] - 10s 10ms/step - loss: 0.1251 - accuracy: 0.9595 - val_loss: 0.1424 - val_accuracy: 0.9528
Epoch 3/50
969/969 [==============================] - 10s 11ms/step - loss: 0.1042 - accuracy: 0.9658 - val_loss: 0.0847 - val_accuracy: 0.9719
Epoch 4/50
969/969 [==============================] - 11s 11ms/step - loss: 0.0924 - accuracy: 0.9695 - val_loss: 0.0821 - val_accuracy: 0.9772
Epoch 5/50
969/969 [==============================] - 11s 11ms/step - loss: 0.0827 - accuracy: 0.9731 - val_loss: 0.0812 - val_accuracy: 0.9742
Epoch 6/50
969/969 [==============================] - 11s 12ms/step - loss: 0.0767 - accuracy: 0.9737 - val_loss: 0.0717 - val_accuracy: 0.9754
Epoch 7/50
969/969 [==============================] - 11s 12ms/step - loss: 0.0669 - accuracy: 0.9782 - val_loss: 0.1018 - val_accuracy: 0.9674
Epoc

In [8]:
class_counts = np.bincount(y_test.astype(int))
print(class_counts)


[    0 58951  2493  6725  2923  1277   425]


In [9]:
from sklearn.metrics import confusion_matrix, f1_score

# Predict the test data
y_pred = mlp.predict(X_test1)

# Convert the predicted probabilities to class labels
y_pred_labels = np.argmax(y_pred, axis=1)

# Create the confusion matrix
cm = confusion_matrix(y_test, y_pred_labels)
print("Confusion Matrix:")
print(cm)

# Calculate the F1 score
f1 = f1_score(y_test, y_pred_labels, average='weighted')
print("F1 Score:", f1)




2275/2275 [==============================] - 4s 2ms/step
Confusion Matrix:
[[58446     2     4   115   303    81]
 [    0  2493     0     0     0     0]
 [   21     0  6614     5    80     5]
 [   57     1     3  2834     3    25]
 [   46     0     3     4  1224     0]
 [    2     0     0     7     0   416]]
F1 Score: 0.9898900605088254
